# Tests

In [1]:
import os
import sys
import logging
import json
import re
import pandas as pd

#dir = os.path.abspath(__file__)
#dir    = os.getcwd()
#pkgdir = os.path.dirname(os.path.dirname(dir)

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib.nanuq import Nanuq
from lib.gapp import Phenotips


In [35]:
os.chdir("D:\\HSJ\\Projects\\PRAGMatIQ\\Runs")
os.listdir()

['README-A00516_0380.ipynb',
 '230131_A00516_0389_AHHHVWDMXY.xlsx',
 'README-A00516_0388.ipynb',
 '230131_A00516_0388_BHJ3KMDMXY.xlsx',
 'README-A00516_0389.ipynb',
 'README-A00516_0385.ipynb',
 'README-A00516_0384.ipynb',
 'README-A00516_0382.ipynb',
 'README-A00516_0376.ipynb',
 'README-A00516_0372.ipynb',
 'README-A00977_0584.ipynb',
 '230207_A00977_0584_BHJ3LYDMXY.xlsx',
 'README-A00516_0392.ipynb',
 '230217_A00516_0392_BH3YJVDSX5.xlsx',
 'README-A00516_0393.ipynb',
 'A00516_0393.json',
 '230224_A00516_0393_AH5WJFDSX5.xlsx',
 'README-A00516_0394.ipynb',
 '230306_A00516_0394_AHJ22WDMXY.xlsx',
 'README-A00516_0397.ipynb',
 'README-A00516_0399.ipynb',
 'README-A00516_0403.ipynb',
 'README-A00516_0404.ipynb',
 'README-A00516_0406.ipynb',
 'README-A00516_0407.ipynb',
 'README-A00516_0411.ipynb',
 'README-A00516_0413.ipynb',
 'README-A00516_0416.ipynb',
 'README-A00516_0420.ipynb',
 'README-A00516_0424.ipynb',
 'archive_PRAGMatIQ.sh',
 'README-A00516_0426.ipynb',
 'README-test_v32.ipynb'

In [3]:
file    = "A00516_0445\\emg_batch_manifest.csv"
def get_samples_list(file):
    samples = []
    with open(file, 'r') as fh:
        for line in fh.readlines():
            cols = line.split(',')
            samples.append(cols[5])
    samples.pop(0)
    return samples
print(get_samples_list(file))


['GM231651', 'GM231626', 'GM231624', 'GM231632', 'GM231627', 'GM231615', 'GM231667', 'GM231668', '23-05982-T1', '23-06384-T1', '23-06383-T1', '3042652455', '3042642360', '3042645886', '3052768938', '3052768961', '3052768951', '3052768942']


In [4]:
nq = Nanuq()
samplenames = nq.get_samplenames("A00516_0445")
samplenames.text

'##2023-08-09\r\n##Centre for Pediatric Clinical Genomics\r\n##Flow Cell: H7N33DSX7\r\n##Principal Investigator: Dr Mehdi Yeganeh, CHUS Service de génétique médicale, Dr Jean-François Soucy\r\n##Nanuq References: A00516_0445\r\n##Content: Internal_Sample_ID -> Client_Sample_Name Conversion grid\r\n##-------------------------------------------\r\n##Internal_Sample_ID\tClient_Sample_Name\r\n##-------------------------------------------\r\n22256\t3042652455\r\n22257\t3042642360\r\n22258\t3042645886\r\n22262\t3052768938\r\n22263\t3052768942\r\n22264\t3052768961\r\n22265\t3052768951\r\n22282\t23-05982-T1\r\n22283\t23-06383-T1\r\n22284\t23-06384-T1\r\n22285\tGM231615\r\n22286\tGM231624\r\n22287\tGM231626\r\n22288\tGM231627\r\n22290\tGM231632\r\n22293\tGM231651\r\n22294\tGM231667\r\n22295\tGM231668\r\n##-------------------------------------------\r\n##Description of the conversion grid:\r\n##A "Client_Sample_Name" represents the name of a sequencing sample initially assigned by the client.\r\

In [33]:
for line in samplenames.text.splitlines():
    print(line)

##2023-08-09
##Centre for Pediatric Clinical Genomics
##Flow Cell: H7N33DSX7
##Principal Investigator: Dr Mehdi Yeganeh, CHUS Service de génétique médicale, Dr Jean-François Soucy
##Nanuq References: A00516_0445
##Content: Internal_Sample_ID -> Client_Sample_Name Conversion grid
##-------------------------------------------
##Internal_Sample_ID	Client_Sample_Name
##-------------------------------------------
22256	3042652455
22257	3042642360
22258	3042645886
22262	3052768938
22263	3052768942
22264	3052768961
22265	3052768951
22282	23-05982-T1
22283	23-06383-T1
22284	23-06384-T1
22285	GM231615
22286	GM231624
22287	GM231626
22288	GM231627
22290	GM231632
22293	GM231651
22294	GM231667
22295	GM231668
##-------------------------------------------
##Description of the conversion grid:
##A "Client_Sample_Name" represents the name of a sequencing sample initially assigned by the client.
##The "Internal_Sample_ID" is the internal identifier assigned by the Center for this sample.
##Use of an "In

In [14]:
fc_date = re.match(r'##(\d{4}-\d{2}-\d{2})', samplenames.text).group(1)
fc_date

'2023-08-09'

In [16]:
for line in samplenames.text.splitlines():
    if not line.startswith('#'):
        cqgc, sample = line.split("\t")
        print(cqgc, sample)


22256 3042652455
22257 3042642360
22258 3042645886
22262 3052768938
22263 3052768942
22264 3052768961
22265 3052768951
22282 23-05982-T1
22283 23-06383-T1
22284 23-06384-T1
22285 GM231615
22286 GM231624
22287 GM231626
22288 GM231627
22290 GM231632
22293 GM231651
22294 GM231667
22295 GM231668


In [31]:
json.loads(nq.get_sample(22283))

[{'ldmSampleId': '23-06383-T1',
  'ldm': 'LDM-CHUS',
  'patient': {'designFamily': 'TRIO',
   'familyId': '23-05982-T1',
   'familyMember': 'FTH',
   'firstName': 'Andre-Philippe',
   'lastName': 'Belley',
   'ramq': 'BELA93092213',
   'sex': 'MALE',
   'mrn': '00000000',
   'ep': 'CHUS',
   'birthDate': '22/09/1993',
   'fetus': False,
   'status': 'UNK'},
  'ldmServiceRequestId': '23-06383-T1',
  'labAliquotId': '22283',
  'panelCode': 'PRAGMATIQ',
  'specimenType': 'NBL',
  'sampleType': 'DNA',
  'ldmSpecimenId': '23-06383-T1'}]

In [34]:
os.getcwd()

'D:\\HSJ\\tmp'

In [40]:
df = pd.read_csv('df.csv')
df

,sample_name,biosample,relation,gender,label,mrn,cohort_type,date_of_birth(YYYY-MM-DD),status,family,case_group_number,phenotypes,hpos,filenames,fc_date
0,GM231651,22293,PROBAND,FEMALE,CHUSJ,02475058,DUO,15/05/2012,AFF,02475058,P0000160,"Premature birth,Abnormal delivery,Caesarian se...","HP:0001622,HP:0001787,HP:0011410,HP:0033454,HP...",/projects/0000000000/biosamples/22293/datasets...,2023-08-09
1,GM231626,22287,MTH,FEMALE,CHUSJ,03443233,DUO,03/02/1980,UNF,02475058,NaN,NaN,NaN,/projects/0000000000/biosamples/22287/datasets...,2023-08-09
2,GM231624,22286,PROBAND,FEMALE,CHUSJ,03439309,TRIO,08/06/2023,AFF,03439309,P0000158,"Poor suck,Maternal hypertension,Poor suck,Decr...","HP:0002033,HP:0008071,HP:0002033,HP:0004325,HP...",/projects/0000000000/biosamples/22286/datasets...,2023-08-09
3,GM231632,22290,MTH,FEMALE,CHUSJ,02171225,TRIO,28/05/1993,UNF,03439309,NaN,NaN,NaN,/projects/0000000000/biosamples/22290/datasets...,2023-08-09
4,GM231627,22288,FTH,MALE,CHUSJ,01956309,TRIO,22/08/1996,UNF,03439309,NaN,NaN,NaN,/projects/0000000000/biosamples/22288/datasets...,2023-08-09
5,GM231615,22285,PROBAND,MALE,CHUSJ,03441802,TRIO,03/12/2021,AFF,03441802,P0000159,"Sensory neuropathy,Motor delay,Areflexia of lo...","HP:0000763,HP:0001270,HP:0002522,HP:0007133,HP...",/projects/0000000000/biosamples/22285/datasets...,2023-08-09
6,GM231667,22294,MTH,FEMALE,CHUSJ,03443697,TRIO,31/07/1994,UNF,03441802,NaN,NaN,NaN,/projects/0000000000/biosamples/22294/datasets...,2023-08-09
7,GM231668,22295,FTH,MALE,CHUSJ,03443698,TRIO,06/06/1980,UNF,03441802,NaN,NaN,NaN,/projects/0000000000/biosamples/22295/datasets...,2023-08-09
8,23-05982-T1,22282,PROBAND,FEMALE,CHUS,1626586,TRIO,24/11/2021,AFF,23-05982-T1,P0000152,"Microcephaly,Ataxia,Plagiocephaly,Developmenta...","HP:0000252,HP:0001251,HP:0001357,HP:0002376,HP...",/projects/0000000000/biosamples/22282/datasets...,2023-08-09
9,23-06384-T1,22284,MTH,FEMALE,CHUS,00000000,TRIO,22/07/1996,UNK,23-05982-T1,NaN,NaN,NaN,/projects/0000000000/biosamples/22284/datasets...,2023-08-09


In [41]:
for index, row in df.iterrows():
    if row['case_group_number'] is None:
        print('is None')
    elif row['case_group_number'] == '':
        print('Empty str')
    elif row['case_group_number'] == 'nan':
        print('is NaN')
    else:
        print(row['case_group_number'])

P0000160
nan
P0000158
nan
nan
P0000159
nan
nan
P0000152
nan
nan
P0000130
nan
nan
P0000132
nan
nan
nan
